# Bain Family Collection Scrapbooks: Rotate Images from List

Need to:
1. some of the images need to be rotated CW/CCW to a vertical page
1. save as uncompressed, 8-bit TIFF tagged with AdobeRGB

In [1]:
# importing
from pathlib import Path
from subprocess import call

from ipywidgets import IntProgress, Label, VBox
from IPython.display import display
from PIL import Image

In [4]:
images_to_rotate_list=[(2, 28), (2, 66), (2, 68), (2, 72), (2, 80), (2, 82), (2, 90), (2, 94)]

In [5]:
# functions
def rotate_bound(image, angle, center=None, scale=1.0):
    # grab the dimensions of the image and then determine the
    # center
    (height, width) = image.shape[:2]
    
    # if the center is None, initialize it as the center of
    # the image
    if center is None:
        centerX = (w // 2)
        centerY = (h // 2)
    else:
        centerX, centerY = center

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((centerX, centerY), -angle, scale)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    width_new = int((height * sin) + (width * cos))
    height_new = int((height * cos) + (width * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (width_new / 2) - centerX
    M[1, 2] += (height_new / 2) - centerY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (width_new, height_new), flags=cv2.INTER_CUBIC)


In [7]:
# batch rotate cw

# progress bar
progress_label = Label(f'{len(images_to_rotate_list)} images to rotate . . .')
progress_bar = IntProgress(min=0, max=len(images_to_rotate_list))
progress_widget = VBox([progress_label, progress_bar])
display(progress_widget)

for index, (box, image) in enumerate(images_to_rotate_list, start=1):
    
    image_path = None
    progress_label.value = image_path.name
    
    output_path = None
    image = Image.open(image_path)
    image.save(output_path, compression=None, dpi=(600., 600.))
    
    # use exiftool to copy ICC profile form original to output image
    cmd = f' exiftool -tagsfromfile {str(image_path)} -ICC_Profile -overwrite_original -q {str(output_path)}'
    call(cmd, shell=True)
    
    progress_bar.value = index
    

AttributeError: 'NoneType' object has no attribute 'name'